In [ ]:
%cd '/content/HW4'

/content/drive/MyDrive/Colab Notebooks/HW4


In [1]:
# imports

from pathlib import Path
from omegaconf import OmegaConf

from src.data_pipeline import CIFAR10Pipeline
from src.runner import ExperimentRunner

In [ ]:
# load config

cfg = OmegaConf.load("conf/config.yaml")

print("="*30)
print("Setting up data pipeline...")
print("="*30)

data_pipeline = CIFAR10Pipeline(cfg)
train_loader, val_loader, test_loader = data_pipeline.setup()

print(f"\nDataset: CIFAR-10")
print(f"Training samples: {len(train_loader.dataset)}")
print(f"Validation samples: {len(val_loader.dataset)}")
print(f"Test samples: {len(test_loader.dataset)}")
print(f"Batch size: {cfg.train.batch_size}\n")

In [3]:
def print_exp_params(cfg):
  model_name = cfg.model.model_name
  print("="*30)
  print(f"Experiment Configuration")
  print("="*30)
  print(f"  Mode: {cfg.mode}")
  print(f"  Model: {cfg.model.model_name}")
  print(f"  Optimizer: {cfg.train.optimizer}")
  print(f"  Learning Rate: {cfg.train.learning_rate}")
  print(f"  Batch Size: {cfg.train.batch_size}")
  print(f"  Epochs: {cfg.train.num_epochs}")
  print(f"  Weight Decay: {cfg.train.weight_decay}")
  if model_name.lower() == 'customcnn':
      print(f"  CNN Layers: {cfg.model.cnn.num_layers}")
      print(f"  Use BN: {cfg.model.cnn.use_batch_norm}")
      print(f"  Use Dropout: {cfg.model.cnn.use_dropout}")
      if cfg.model.cnn.use_dropout:
        print(f"  Dropout Rate: {cfg.model.cnn.dropout_rate}")
      print(f"  FC Layers: {cfg.model.cnn.fc_layers}")

In [ ]:
# initialize experiment params

cifar10_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                   'dog', 'frog', 'horse', 'ship', 'truck']

# Dictionary to store results from all experiments
experiment_results = {}

# Run experiment for the configured model
print_exp_params(cfg)


In [ ]:
# run single experiment

# define experiment name as you wish
exp_name = cfg.model.model_name

runner = ExperimentRunner(
    cfg=cfg,
    exp_name=exp_name,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    class_names=cifar10_classes
)

results = runner.run()

experiment_results[results['model_name']] = results

print("\n" + "="*30)
print("Experiment completed successfully!")
print("="*30)

In [ ]:
# Load and evaluate existing model weights

# Example: Load weights using config
# Set cfg.load_weights = True and cfg.mode to match the model you want to load
cfg.load_weights = True
cfg.mode = 'pretrained'  # or 'scratch'
cfg.train.freeze = False  # If loading pretrained model, set freeze accordingly

# Create runner with load_weights=True
runner = ExperimentRunner(
    cfg=cfg,
    exp_name=cfg.model.model_name,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    class_names=cifar10_classes,
    load_weights=True
)

# Check if weights file exists
print(f"Model path: {runner.weights_path}")
print(f"Model exists: {Path(runner.weights_path).exists()}")

# Run evaluation (will skip training and load weights)
if Path(runner.weights_path).exists():
    results = runner.run()
    experiment_results[results['model_name']] = results

    print("\n" + "="*60)
    print("Model evaluation completed!")
    print("="*60)
else:
    print(f"\nNo saved model found at {runner.weights_path}")
    print("Train a model first or check your config parameters.")

## Experiments

We perform the following experiments for each architecture (Shallow vs Deep)
1. Architecture Params
1. Batch Size (64 vs 128)
2. Optimizer & lr (adam vs SGD, 1e-3 vs 1e-4)
3. Regularization (Dropout vs BN vs Both)

In [ ]:
# define 1st experiment params
experiments = [
    
    # Shallow CNN from scratch

    # Shallow_convs=[64]_fc=[256]_bz=128_opt=adam_lr=1e-3
    {
        'exp_name': 'Shallow_convs=[64]_fc=[256]_bz=128_opt=adam_lr=1e-3',
        'model_name': 'CustomCNN',
        'mode': 'scratch',
        'load_weights': False,
        'cnn': {
          'num_layers': [64],
          'kernel_size': [3],
          'stride': [1],
          'fc_layers': [256],
          'use_dropout': False,
          'dropout_rate': 0.4,
          'use_batch_norm': False
        },
        'train': {
          'batch_size': 128,
          'learning_rate': 0.001,
          'num_epochs': 20,
          'optimizer': 'adam',
          'weight_decay': 0.0001,
          'momentum': 0.9,
          'early_stopping_patience': 3,
          'freeze': 'false'
        }
    },

    # Shallow_convs=[64, 128]_fc=[256, 128]_bz=128_opt=adam_lr=1e-3
    {
        'exp_name': 'Shallow_convs=[64, 128]_fc=[256, 128]_bz=128_opt=adam_lr=1e-3',
        'model_name': 'CustomCNN',
        'mode': 'scratch',
        'load_weights': False,
        'cnn': {
          'num_layers': [64, 128],
          'kernel_size': [3, 3],
          'stride': [1, 1],
          'fc_layers': [256, 128],
          'use_dropout': False,
          'dropout_rate': 0.4,
          'use_batch_norm': False
        },
        'train': {
          'batch_size': 128,
          'learning_rate': 0.001,
          'num_epochs': 20,
          'optimizer': 'adam',
          'weight_decay': 0.0001,
          'momentum': 0.9,
          'early_stopping_patience': 3,
          'freeze': 'false'
        }
    },

    # Deep CNN from scratch

    # Deep_convs=[32, 64, 128, 256]_fc=[512, 128]_bz=128_opt=adam_lr=1e-3
    {
        'exp_name': 'Deep_convs=[32, 64, 128, 256]_fc=[512, 128]_bz=128_opt=adam_lr=1e-3',
        'model_name': 'CustomCNN',
        'mode': 'scratch',
        'load_weights': False,
        'cnn': {
          'num_layers': [32, 64, 128, 256],
          'kernel_size': [3, 3, 3, 3],
          'stride': [1, 1, 1, 1],
          'fc_layers': [512, 128],
          'use_dropout': False,
          'dropout_rate': 0.4,
          'use_batch_norm': False
        },
        'train': {
          'batch_size': 128,
          'learning_rate': 0.001,
          'num_epochs': 20,
          'optimizer': 'adam',
          'weight_decay': 0.0001,
          'momentum': 0.9,
          'early_stopping_patience': 3,
          'freeze': 'false'
        }
    },    

    # Deep_convs=[32, 64, 128, 256]_fc=[512, 128]_bz=128_opt=adam_lr=1e-3
    {
        'exp_name': 'Deep_convs=[32, 64, 64, 128, 256]_fc=[512, 256, 128]_bz=128_opt=adam_lr=1e-3',
        'model_name': 'CustomCNN',
        'mode': 'scratch',
        'load_weights': False,
        'cnn': {
          'num_layers': [32, 64, 64, 128, 256],
          'kernel_size': [3, 3, 3, 3, 3],
          'stride': [1, 1, 1, 1, 1],
          'fc_layers': [512, 256, 128],
          'use_dropout': False,
          'dropout_rate': 0.4,
          'use_batch_norm': False
        },
        'train': {
          'batch_size': 128,
          'learning_rate': 0.001,
          'num_epochs': 20,
          'optimizer': 'adam',
          'weight_decay': 0.0001,
          'momentum': 0.9,
          'early_stopping_patience': 3,
          'freeze': 'false'
        }
    },   
]


In [ ]:
# run multiple experiments

for exp in experiments:
    cfg.mode = exp['mode']
    cfg.model.model_name = exp['model_name']
    if 'train' in exp:
      for key in exp['train'].keys():
        cfg['train'][key] = exp['train'][key]
    if 'cnn' in exp:
      for key in exp['cnn'].keys():
        cfg['model']['cnn'][key] = exp['cnn'][key]

    # reload data for new config
    data_pipeline = CIFAR10Pipeline(cfg)
    train_loader, val_loader, test_loader = data_pipeline.setup()
    
    print_exp_params(cfg)

    runner = ExperimentRunner(
        cfg=cfg,
        exp_name=exp['exp_name'],
        train_loader=train_loader,
        val_loader=val_loader,
        test_loader=test_loader,
        class_names=cifar10_classes,
        load_weights=exp['load_weights']
    )

    results = runner.run()
    exp_name = results['model_name']
    experiment_results[exp_name] = results

# Compare all experiments
ExperimentRunner.compare(experiment_results, save_dir="results")